In [34]:
import pandas as pd
from io import StringIO
import googlemaps
import folium
import os
import folium
import ast
import textwrap
import googlemaps
from folium.plugins import Fullscreen, MiniMap
from branca.element import Template, MacroElement


In [2]:
csv_string = '''Name,Party,Address,Rent_or_own,Cost
"Adrienne Adams","Democratic","144-20 109th Avenue, Jamaica","Own","$85,000 in 1987, per her campaign; comparable homes nearby are selling for between $600,000 and over $1 million"
"Andrew Cuomo","Democratic","420 East 54th Street","Rent","Either $7,800 or “about $8,000”"
"Michael Blake","Democratic","837 Washington Avenue, Melrose","Own","$440,000 in 2021; pays $1,800 a month in additional fees"
"Zohran Mamdani","Democratic","32-15 35th Street, Astoria","Rent","$2,300 a month"
"Scott Stringer","Democratic","25 Broad Street","Rent","$6,400 a month"
"Brad Lander","Democratic","256 13th Street, Park Slope","Own","$655,000 in 2002; pays $3,300 a month in mortgage fees; comparable units sold for between $2.4 million and $2.7 million this year"
"Jessica Ramos","Democratic","37-15 79th Street, Jackson Heights","Rent","$2,480 a month, per the campaign"
"Zellnor Myrie","Democratic","10 Maple Street, Prospect–Lefferts Gardens","Rent","$1,300 a month, per the campaign"
"Whitney Tilson","Democratic","1165 Fifth Avenue","Own","$2.1 million in 2002; pays around $5,000 in monthly maintenance fees and taxes; comparable units are around $2.7 million now with nearly $6,000 in monthly maintenance fees"
"Jim Walden","Independent","200 Hicks Street, Brooklyn Heights","Own","$1.1 million in 2009; comparable unit sold for $3.3 million in 2021 with $4,843 in monthly maintenance fees"
"Curtis Sliwa","Republican","272 West 73rd Street","Rent","$3,675 a month"'''

In [6]:
df = pd.read_csv(StringIO(csv_string))

In [9]:
%store -r google_maps_API_Key
gmaps_key = googlemaps.Client(key=google_maps_API_Key)

%store -r map_box_api_key

In [10]:
# Define the geocode function
def geocode(add):
    g = gmaps_key.geocode(add)
    if g:
        lat = g[0]["geometry"]["location"]["lat"]
        lng = g[0]["geometry"]["location"]["lng"]
        return (lat, lng)
    else:
        return None

# Apply geocoding to the 'geo_address' column and store the results in 'geocoded' column
df['geocoded'] = df['Address'].apply(geocode)

In [19]:
df['geocoded'] = df['geocoded'].astype(str)
df[['lat', 'lon']] = df['geocoded'].apply(lambda x: (None, None) if x == 'None' else x.strip('()').split(', ', 1)).apply(pd.Series)
df['lat'] = df['lat'].astype(float)
df['lon'] = df['lon'].astype(float)

In [20]:
map_df = df

In [17]:
map_df.columns

Index(['Name', 'Party', 'Address', 'Rent_or_own', 'Cost', 'geocoded'], dtype='object')

In [32]:
party_colors = {
    "Democratic": "blue",
    "Republican": "red",
    "Independent": "green"
    # Add more parties here if needed, or default to 'gray'
}

In [39]:
# 1) Popup HTML
def create_popup(row):
    lines = [
        f"<h3>{row['Name']}</h3>",
        f"<strong>{row['Party']}</strong><br>",
        f"<strong>{row['Address']}</strong><br>"
    ]
    return "<div class='popup-content'>\n" + "\n".join(lines) + "\n</div>"

# 2) Initialize map
first_lat = float(map_df['lat'].iloc[1])
first_lon = float(map_df['lon'].iloc[1])
m = folium.Map(
    location=[first_lat, first_lon],
    zoom_start=10.5,
    scrollWheelZoom=False
)

# 3) Mapbox tile layer
folium.TileLayer(
    tiles=(
        f"https://api.mapbox.com/styles/v1/"
        f"mapbox/streets-v11/tiles/256/{{z}}/{{x}}/{{y}}@2x"
        f"?access_token={map_box_api_key}"
    ),
    attr="Mapbox",
    name="Streets",
    overlay=True,
    control=False,
    show=False,
    min_zoom=1,
    max_zoom=20
).add_to(m)

# 4) Optional CSS & Title
custom_css = """
<style>
  .popup-content { min-width:300px; font-size:14px; color:#333; }
  .leaflet-popup-content-wrapper { background:#f9f9f9; border:1px solid #bbb; }
  .leaflet-popup-tip { background:#f9f9f9; }
</style>
"""
m.get_root().html.add_child(folium.Element(custom_css))

title_html = """
<h3 style="text-align:center;
           font-family:Arial,sans-serif;
           font-size:18px;
           color:#333;
           margin-top:10px;">
  <b>Projects seeking 485x</b>
</h3>
"""
m.get_root().html.add_child(folium.Element(title_html))

# 6) Fixed radius for all bubbles
fixed_radius = 8  # pixels; tweak as needed

title_html = """
{% macro html(this, kwargs) %}
  <div style="
      position: fixed;
      top: 10px;
      left: 50%;
      transform: translateX(-50%);
      background-color: white;
      padding: 6px 12px;
      border: 2px solid grey;
      z-index:9999;
      font-size:16px;
      font-weight: bold;
      ">
    Residences of NYC Mayoral Candidates
  </div>
{% endmacro %}
"""
title = MacroElement()
title._template = Template(title_html)
m.get_root().add_child(title)


# ─────────────────────────────────────────────────────────────────────────────
# 7) Add map controls (Fullscreen button, minimap)
# ─────────────────────────────────────────────────────────────────────────────
Fullscreen().add_to(m)


# ─────────────────────────────────────────────────────────────────────────────
# 8) Add CircleMarkers for each row, color-coded by party
# ─────────────────────────────────────────────────────────────────────────────
fixed_radius = 8  # radius in pixels for each CircleMarker

for _, row in map_df.iterrows():
    lat, lon = float(row["lat"]), float(row["lon"])
    popup = folium.Popup(create_popup(row), max_width=220)

    # Look up the party color, default to gray if party not found
    clr = party_colors.get(row["Party"], "gray")

    folium.CircleMarker(
        location=[lat, lon],
        radius=fixed_radius,
        color=clr,
        fill=True,
        fill_color=clr,
        fill_opacity=0.6,
        weight=1,
        popup=popup
    ).add_to(m)

m

In [40]:
m.save('index.html')

In [41]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/nyc_residents
